<a href="https://colab.research.google.com/github/RealebohaRamphielo/CA1_PDA/blob/main/Realeboha_Ramphielo(10622234)_CA2_Programming_for_Data_Analysis_B9AI108_Dec22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><b>Realeboha Raymond Ramphielo (10622234)</b></h1>

<h3><i> B9AI108 Programming For Data Analysis</h3>
<h3> CA 2 </i></h3>

<ul type=None>
<li><a href="https://github.com/RealebohaRamphielo/CA1_PDA"> GitHub link </a></li>
<li><a href="https://colab.research.google.com/drive/1XIZa4PnihdmvbuFt9s89LH5AF3aYZIFr?usp=sharing"> Colab Link </a></li>
</ul>
<p align=justify>Please remember to open in jupyter as colab might bring some issues when getting/ fetching the url. The project is hosted in colab, so regular updates are on colab, a copy was saved to GitHub for backup purposes, no real work was done on GitHub</p>

## **Intoduction**
<p align=justify> Job scraping is the application of web scraping to employment. Users can easily collect employment information and build a meaningful job database using this automated online data extraction technique. There are several websites to fetch job data, these range from big job boards like <i>Irishjobs.ie</i> and the likes, right down to individual companies’ career portals. Jobs scraping involves gathering information regarding jobs and downloading it into structured and usable formats, like CSV, that can be used for analysis (Gangadhara Reddy and Viswanath, 2022). An obvious advantage to job scraping is probably data relevancy and timeliness. I will capture the HTML text using the standard Requests and Beautiful Soup configurations, and then convert it to a Beautiful Soup object to simply parse through it and retrieve the data points. I plan to scrap the Career Junction website (South African). From the website, I will scrap the Job title, copany, location, salary, day of posting, day of expiry, and a link to the job. </p>  


## Libarary intallations

In [ ]:
pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install bs4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime, date, timedelta 
import numpy as np

## **Steps**
<ol>
<li>Getting the url</li>
<li>Scaling the salary</li>
<li>Maing the soup</li>
<li>Creating the dataframe and csv</li>
</ol>

### ***Getting the url*** 
<p align=justify> The get_url function will accept two string parameters; job and location, location is set at default value of 0 which translates to all locations. Within this function is a dictionary variable, locations. Locations will convert/ encode the second parameter into a usable location code by pointing to the location's value in the locations deictionary. After encoding the location, the next step is to 'plug' the desired job title and encoded location into the url and create a url that will be returned by the function. In a case where the passed location is not defined, a KeyValueError will be raised by the function and an error message will be returned. </p>

In [ ]:
def get_url(job, location = 0):
  
  '''This function sets up the search url from the user defines job and location,
    default location will be 0'''

  #When no location is specified, default location = 0, i.e. the search will not
  # have any location restrictions

  locations = {'Gauteng':'2747', 'Western Cape':'16149', 'KwaZulu-Natal':'13131',\
              'Eastern Cape':'2', 'Free State':'1782', 'Mpumalanga':'14867',\
              'Limpopo':'14594', 'North West':'15372', 'Northern Cape':'15837',\
              'International':'1000001', 'Working From Home':'100000'}

  #above is the locations dictionary that will encode the location from a normal
  #word search to an 'int' search
  
  error_message = 'No access to location, please revise'
  
  try:
    if location != 0:                    #encode the location
      location = locations[location]
    gen_url = "https://www.careerjunction.co.za/jobs/results?keywords={}&autosuggestEndpoint=%2Fautosuggest&location={}&category=&btnSubmit=+"
    search_url = gen_url.format(job, location)
    
    return search_url
  
  except KeyError:
    return error_message

###***Scaling the salary***
<p align=justify> The salary_convert funtion will take one string parameter. The purpose of this function is to scale the salary to monthly rates. It will first strip the string off the 'R' which denotes the ZAR (South AFrican Rand), then strip off any white and split the string using white space. Following the split, we will take the first element (element at index 0) and follow this by splitting it using the period (dot) character and assign to a_string variable the first element from the resulting split. Some figures are comma separated, thus the second step will be to remove any commas from a_string and covert the variable to a float and multiply it by the appropriate scaling factor to calculate a monthly salary, the scaling factors are as follows:
<ul>
<li>160 for hourly pay</li>
<li>20 for daily pay</li>
<li>4 for weekly pay</li>
<li>2 for fortnight pay</li>
<li>1 for monthly pay</li>
<li>1/12 for annual pay</li>
</ul>
The function will return the scaled salary after multiplying it by the scaling factor. In instances where the salary is not mentioned/ disclosed (or when no figure is given), the figure will return a NaN value (missing value) from Numpy.
</p>

In [ ]:
def salary_convert(b_string):

  ''' The function will convert help extract salaries and scales them to a 
    monthly value'''

  if 'per hour' in b_string:          #hourly rate
    a_string = b_string.strip('R').strip().split()[0].split('.')[0] 
    a_string = float(a_string.replace(',', '')) * 160

  elif 'per day' in b_string:         #daily rate
    a_string = b_string.strip('R').strip().split()[0].split('.')[0]
    a_string = float(a_string.replace(',', '')) * 20

  elif 'per week' in b_string:         #weekly rate
    a_string = b_string.strip('R').split()[0].split('.')[0]
    a_string = float(a_string.replace(',', '')) * 4

  elif 'per fort' in b_string:         #fortnight rate
    a_string = b_string.strip('R').strip().split()[0].split('.')[0]
    a_string = float(a_string.replace(',', '')) * 2

  elif 'per month' in b_string:        #monthly rate
    a_string = b_string.strip('R').strip().split()[0].split('.')[0]
    a_string = float(a_string.replace(',', ''))

  elif 'per year' in b_string:       #yearly rate
    a_string = b_string.strip('R').strip().split()[0].split('.')[0]
    a_string = float(a_string.replace(',', '')) / 12

  else:    #Undisclosed salaries
    a_string = np.NaN

  return a_string

### ***Making the Soup***
<p align=justify>
The make_soup function will take job (string) and location (string, but has a default 0 value) parameters. The first step is to pass these parameters to the get_url function and create a url. This will be followed by adding a page number to the url and creating a request. The request content; response will be be passed through Beautiful Soup which will help us create the Soup variable that we will call pasta. Jobs will be set as the jobs container part of the pasta and individual job cards will be stored as j_cards. We will loop over the j_cards list, every job card will have its dictionary that will later be added to the job collection list following each iteration. A job title, company, location, salary, posting date, expiry date and link will all be added as Key and value pairs to the individual job dictionary. Monthly salary will be calculated by calling the salary_convert function and added to the dictionary. The duration of the ad in days from day of posting to expiry date will be calculated within the for loop by subtracting the posting date from the expiry date. Also calculated within the for loop, are; the days left before the ad expires as the difference between the expiry date and the day of extraction (scraping). Both these will be added as individual key-value pairs to the dictionary. This process will continue until the needed information from each job on one page is collected, the process will continue for every page until the last page, which will be set to a maximum of 300 paged due to some shortcomings. make_soup will return job_collection, a list of dictionaries of every scraped job.
</p>

In [ ]:
def make_soup(job, location=0):
  
  '''The function creates a response from the created by get_url, then scraps 
    the actual data from the website'''

  gen_url = get_url(job, location)    #defines the website that will be scraped
  job_collection = []    #empty list to contain all the extracted job attributes 

  for i in range(1,300):
    search_page = f"&page={str(i)}"          #this is the start page
    url = gen_url + search_page             #add a page to the url
    the_response = requests.get(url)  

    pasta = BeautifulSoup(the_response.content, 'html.parser') #create the soup
    jobs = pasta.find('div', class_="two-thirds")          #location of the jobs on the page
    try:
      j_cards = jobs.find_all('div', class_="job-result")    #collection of the jobs on the page
    
      for j_card in j_cards:  #looping over the job cards list to extract features of each job
        a_job = {}            #dictionary to hold individual job information
        
        prim_link = "https://www.careerjunction.co.za"
        j_today = date.today()
        j_title = j_card.find('h2')
        
        a_job['Job Title'] = j_title.text.strip()
        if j_card.find('h3') is not None:
          a_job['Company'] = j_card.find('h3').text.strip()
        a_job['Location'] = j_card.find('li', class_="location").text.strip()
        a_job['Job Type'] = j_card.find('li', class_="position").text.strip()

        a_job['Salary'] = j_card.find('li', class_="salary").text.strip()
        a_job['Monthly Salary'] = salary_convert(a_job['Salary'])

        a_job['Posting Date'] = j_card.find('li', class_="updated-time").text\
        .strip('Posted').strip()

        j_expiry = j_card.find('li', class_="expires").text.split()[2]
        try:
          ini_exp = j_today + timedelta(days=int(j_expiry))
        except ValueError:
          ini_exp = j_today
        a_job['Expiry Date'] = ini_exp.strftime('%d %b %Y')
        j_duration = pd.to_datetime(a_job['Expiry Date'])-pd.to_datetime(a_job['Posting Date'])
        j_toExpire = pd.to_datetime(a_job['Expiry Date'])-pd.to_datetime(j_today.strftime('%d %b %Y'))
        a_job['Ad Duration (Days)'] = int(str(j_duration).split()[0])
        a_job['Days before expiry'] = int(str(j_toExpire).split()[0])

        a_job['Link'] = prim_link + j_title.find('a').get('href')

        job_collection.append(a_job) 
        #adds the single jobs dictionary to the jobs collection list, feature names are the keys
    except AttributeError:
      break    #this break the function if it has run out of pages to scrap

  return job_collection    #returns jobs list


### ***Creating a data frame and CSV file***

<p align=justify>
The last function, main_jobs, will take three parameters, job, location (with default value of zero), and the file_name; with defualt value 'your_file'. This function calls make_soup and passes the first two arguments. The resulting list of dictionaries will be converted to a pandas dataframe, then the columns will be rearranged for easier browsing following which, a CSV will be written from the resulting dataframe, it will take its name from the third parameter. The function also returns this dataframe. 
</p>

In [ ]:
def main_jobs(job, location=0, file_name='your_file'):

  '''Runs make_soup functions then creates a pandas dataframe that will be
    converted into a csv file'''
    
  cols_ord = ['Job Title', 'Location', 'Job Type', 'Company', 'Salary', \
              'Monthly Salary', 'Posting Date', 'Expiry Date', \
              'Ad Duration (Days)','Days before expiry', 'Link']

  j_data = make_soup(job, location)  #defines a list of dictionaries
  df = pd.DataFrame(j_data)          #coverts the list above to a dataframe
  df= df[cols_ord]                   #reorders the columns
  df.to_csv(file_name+'.csv')   #creates a jobs_data csv file from the dataframe

  return df 

### ***Some tests***

In [ ]:
some_some = main_jobs('engineer', 'Western Cape', file_name='engineering jobs')
some_some2 = main_jobs('data', file_name='data jobs')
some_some3 = main_jobs('manager', 'Gauteng', file_name='managerial jobs')


In [ ]:
somerrrr = main_jobs('developer')

In [ ]:
print(some_some['Ad Duration (Days)'].max(), some_some['Ad Duration (Days)']\
      .min(), round(some_some['Ad Duration (Days)'].mean()))

In [ ]:
# #from numpy.lib.shape_base import dsplit
# li = []
# for i in range(1,3):
#   gen_url3 = "https://www.careerjunction.co.za/jobs/results?keywords=data&autosuggestEndpoint=%2fautosuggest&location=2747&category=&btnSubmit=+"
#   #url1 = get_url('data', 'Limpopo')
#   cococ = f"&page={str(i)}"
#   gen_url = gen_url3 + cococ
#   response = requests.get(gen_url)
#   resp=response.content
#   #print(resp, resp.reason)
#   #print(gen_url)
#   #print(url1)
#   #resp.status_code
#   pasta = BeautifulSoup(resp, 'html.parser')
#   jobs = pasta.find('div', class_="two-thirds")
#   j_cards = jobs.find_all('div', class_="job-result")
#   #j_cards.pop(5)
#   print(len(j_cards))
#   #j_cards
#   #li = []
#   for j_card in j_cards:
#     j_today = date.today()
#     a_job = {}
#     titleofjob = j_card.find('h2')
#     a_job['Job Title'] = (titleofjob.text.strip())
#   #  a_job['Company'] = (j_card.find('h3').text.strip())
#     a_job['Location'] = (j_card.find('li', class_="location").text.strip())
#     a_job['Job Type'] = (j_card.find('li', class_="position").text.strip())
#     a_job['Salary'] = (j_card.find('li', class_="salary").text.strip())
#     a_job['Monthly Salary'] = salary_convert(a_job['Salary'])
    
#     a_job['Posting date'] = (j_card.find('li', class_="updated-time").text.strip('Posted').strip())
#     j_expiry = j_card.find('li', class_="expires").text.split()[2]
#     if j_expiry != 'today':
#       ini_exp = j_today + timedelta(days=int(j_expiry))
#       a_job['Expiry date'] = ini_exp.strftime('%d %b %Y')
#     else:
#       a_job['Expiry date'] = j_today.strftime('%d %b %Y')
#     #a_job['Job URL'] = (j_card.find('h2').text.strip())
#     duration1 = pd.to_datetime(a_job['Expiry date']) - pd.to_datetime(a_job['Posting date'])
#     duration2 = pd.to_datetime(a_job['Expiry date']) - pd.to_datetime(j_today.strftime('%d %b %Y'))
#     #a_job['j_title'] = (j_card.find('h2').text.strip())
#     a_job['sahdis'] = int(str(duration1).split()[0])
#     a_job['sahdis2'] = int(str(duration2).split()[0])
#     a_job['Link']  = 'https://www.careerjunction.co.za'+titleofjob.find('a').get('href')

#     li.append(a_job)

#   #two-way step in accessing an embedded feature, find outer tag and assign it
#   #then use asigned_varaible.attrs['some_tag'] to locate the embedded feature
#   #eg -> var = j_card.find('h2') followed by feature = var.attrs['a'] 

#   #convert final list of dictionaries to a pandas dataframe then use to_csv after
#   #df = pd.DataFrame(jobs) followed by df.to_csv('file_name.csv')

#   #len(li)
#   #li[3]
# #li


In [ ]:
gen_url100 = "https://www.careerjunction.co.za/jobs/results?keywords=data&autosuggestEndpoint=%2fautosuggest&location=2747&category=&btnSubmit=+&page=2898"
resp100 = requests.get(gen_url100)
print(resp100, resp100.reason) 
print(resp100.status_code)

In [ ]:
# li = []
# for j_card in j_cards:
#   j_today = date.today()
#   a_job = {}
#   a_job['Job Title'] = (j_card.find('h2').text.strip())
#   a_job['Company'] = (j_card.find('h3').text.strip())
#   a_job['Location'] = (j_card.find('li', class_="location").text.strip())
#   a_job['Job Type'] = (j_card.find('li', class_="position").text.strip())
#   a_job['Salary'] = (j_card.find('li', class_="salary").text.strip('R').strip())
#   a_job['Posting date'] = (j_card.find('li', class_="updated-time").text.strip('Posted').strip())
#   j_expiry = j_card.find('li', class_="expires").text.split()[2]
#   if j_expiry != 'today':
#     ini_exp = j_today + timedelta(days=int(j_expiry))
#     a_job['Expiry date'] = ini_exp.strftime('%d %b %Y')
#   else:
#     a_job['Expiry date'] = j_today.strftime('%d %b %Y')
#   a_job['Duration'] = pd.to_datetime(a_job['Expiry date']) - pd.to_datetime(a_job['Posting date'])
#   #a_job['Job URL'] = (j_card.find('h2').text.strip())
#   #a_job['j_title'] = (j_card.find('h2').text.strip())

#   li.append(a_job)

# #two-way step in accessing an embedded feature, find outer tag and assign it
# #then use asigned_varaible.attrs['some_tag'] to locate the embedded feature
# #eg -> var = j_card.find('h2') followed by feature = var.attrs['a'] 

# #convert final list of dictionaries to a pandas dataframe then use to_csv after
# #df = pd.DataFrame(jobs) followed by df.to_csv('file_name.csv')

# len(li)
# li[3]

### Attempting wrapping inside a class

In [52]:
class JobScraping():
  def __init__(self, job, locs=0, file_name='your_file'):
    self.locations = {'Gauteng':'2747', 'Western Cape':'16149', 'KwaZulu-Natal':'13131',\
                      'Eastern Cape':'2', 'Free State':'1782', 'Mpumalanga':'14867',\
                      'Limpopo':'14594', 'North West':'15372', 'Northern Cape':'15837',\
                      'International':'1000001', 'Working From Home':'100000'}
    # above is the locations dictionary that will encode the location from a 
    # normal word search to an 'int' search
    self.job = job
    self.locs = locs
    self.file_name = file_name
    self.the_url = self._get_url
    self.the_soup = self._soup_set
    #self.the_file = self._main_run

  def _get_url(self):
    '''This method sets up the search url from the user defines job and 
    location, default location will be 0'''

    # When no location is specified, default location = 0, i.e. the search will 
    # not have any location restrictions
 
    error_message = 'No access to location, please revise'
    
    try:
      if self.locs != 0:                    #encode the location
        self.locs = self.locations[self.locs]
      gen_url = "https://www.careerjunction.co.za/jobs/results?keywords={}&autosuggestEndpoint=%2Fautosuggest&location={}&category=&btnSubmit=+"
      search_url = gen_url.format(self.job, self.locs)
      
      return search_url
    
    except KeyError:
      return error_message

  def sal_set(self, sal):
      ''' The method will convert help extract salaries and scales them to a 
      monthly value'''

      if 'per hour' in sal:    #hourly rate
        a_string = sal.strip('R').strip().split()[0].split('.')[0] 
        a_string = float(a_string.replace(',', '')) * 160

      elif 'per day' in sal:         #daily rate
        a_string = sal.strip('R').strip().split()[0].split('.')[0]
        a_string = float(a_string.replace(',', '')) * 20

      elif 'per week' in sal:         #weekly rate
        a_string = sal.strip('R').split()[0].split('.')[0]
        a_string = float(a_string.replace(',', '')) * 4

      elif 'per fort' in sal:         #fortnight rate
        a_string = sal.strip('R').strip().split()[0].split('.')[0]
        a_string = float(a_string.replace(',', '')) * 2

      elif 'per month' in sal:        #monthly rate
        a_string = sal.strip('R').strip().split()[0].split('.')[0]
        a_string = float(a_string.replace(',', ''))

      elif 'per year' in sal:       #yearly rate
        a_string = sal.strip('R').strip().split()[0].split('.')[0]
        a_string = float(a_string.replace(',', '')) / 12

      else:    #Undisclosed salaries
        a_string = np.NaN

      return a_string

  def _soup_set(self):
    '''The method creates a response from the created by get_url, then scraps 
    the actual data from the website''' #self.job, self.locs

    gen_url = self._get_url()    #defines the website that will be scraped
    job_collection = []    #empty list to contain all the extracted job attributes 

    for i in range(1,200):
      search_page = f"&page={str(i)}"          #this is the start page
      url = gen_url + search_page             #add a page to the url
      the_response = requests.get(url)  

      pasta = BeautifulSoup(the_response.content, 'html.parser') #create the soup
      jobs = pasta.find('div', class_="two-thirds")          #location of the jobs on the page
      try:
        j_cards = jobs.find_all('div', class_="job-result")    #collection of the jobs on the page
      
        for j_card in j_cards:  #looping over the job cards list to extract features of each job
          a_job = {}            #dictionary to hold individual job information
          
          prim_link = "https://www.careerjunction.co.za"
          j_today = date.today()
          j_title = j_card.find('h2')
          
          a_job['Job Title'] = j_title.text.strip()
          if j_card.find('h3') is not None:
            a_job['Company'] = j_card.find('h3').text.strip()
          a_job['Location'] = j_card.find('li', class_="location").text.strip()
          a_job['Job Type'] = j_card.find('li', class_="position").text.strip()

          a_job['Salary'] = j_card.find('li', class_="salary").text.strip()
          a_job['Monthly Salary'] = self.sal_set(a_job['Salary'])

          a_job['Posting Date'] = j_card.find('li', class_="updated-time").text\
          .strip('Posted').strip()

          j_expiry = j_card.find('li', class_="expires").text.split()[2]
          try:
            ini_exp = j_today + timedelta(days=int(j_expiry))
          except ValueError:
            ini_exp = j_today
          a_job['Expiry Date'] = ini_exp.strftime('%d %b %Y')
          j_duration = pd.to_datetime(a_job['Expiry Date'])-pd.to_datetime(a_job['Posting Date'])
          j_toExpire = pd.to_datetime(a_job['Expiry Date'])-pd.to_datetime(j_today.strftime('%d %b %Y'))
          a_job['Ad Duration (Days)'] = int(str(j_duration).split()[0])
          a_job['Days before expiry'] = int(str(j_toExpire).split()[0])

          a_job['Link'] = prim_link + j_title.find('a').get('href')

          job_collection.append(a_job) 
          #adds the single jobs dictionary to the jobs collection list, feature names are the keys
      except AttributeError:
        break    #this break the function if it has run out of pages to scrap

    return job_collection    #returns jobs list


  def main_run(self):
      '''Runs make_soup functions then creates a pandas dataframe that will be
      converted into a csv file'''
      
      cols_ord = ['Job Title', 'Location', 'Job Type', 'Company', 'Salary', \
                  'Monthly Salary', 'Posting Date', 'Expiry Date', \
                  'Ad Duration (Days)','Days before expiry', 'Link']

      j_data = self.the_soup()  #defines a list of dictionaries
      df = pd.DataFrame(j_data)          #coverts the list above to a dataframe
      df= df[cols_ord]                   #reorders the columns
      df.to_csv(self.file_name+'.csv')   #creates a jobs_data csv file from the dataframe

      #self.job, self.locs

      return df 

In [53]:
manager_jobs = JobScraping('manager', locs="Limpopo", file_name='managerjobs2')
mngr_file = manager_jobs.main_run()


In [ ]:
mngr_file

In [ ]:
mngr_file[mngr_file['Days before expiry']<10]

In [ ]:
mngr_file[mngr_file['Days before expiry'] == 20]